# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Turaev cobraket

In [5]:
def ep(h1, h2) # h1 = {letter: *, index: *}
  result = if h1[:letter].show == h2[:letter].show
    if (h1[:index] < h2[:index]) 
#    if (h1[:label] < h2[:label]) 
      (h2[:label].odd?) ? -1 : 1
    else
      (h1[:label].odd?) ? 1 : -1
    end
  else
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  end
  return result
end
#--------------------------------------------
def lk(word, div_pair)
  word.flatten!
  hash_of_letters = div_pair.map do |k|
    [{index: k%(word.size)+1, letter: word[k%(word.size)], label: 2*(k%(word.size))},
       {index: k, letter: word[k-1].inverse, label: 2*k-1}]
  end
  

  if (hash_of_letters[0][0][:letter] == hash_of_letters[0][1][:letter]) or (hash_of_letters[1][0][:letter] == hash_of_letters[1][1][:letter])
    return 0
  else
    terms = [[0,0],[0,1],[1,0],[1,1]].map do |a|
        [hash_of_letters[0][a[0]], hash_of_letters[1][a[1]]]
    end
    vals = terms.map{|v| ep(*v)}
    return (-1)*(vals[0] - vals[1] - vals[2] + vals[3])/2
#    return [terms.map{|a| "(#{a.map{|h| h[:letter].show + ':' + h[:index].to_s + ':' + h[:label].to_s}.join(',')})"}.join(','), vals, (-1)*(vals[0] - vals[1] - vals[2] + vals[3])/2]
  end
end
nil

In [21]:
def cobracket(myw, verbose: false)
  
  #-----
  # 2つの div の組み合わせ全てについて、{divs: *, linking: *, words: *} という Hash を生成。
  
  divs = (1..myw.size).to_a
  monomials = divs.combination(2).map do |combi|
    #--- dividng word --- 
    facs = myw.flatten.factors
    words = Array.new(2)
    words[0] = Word.new(facs[0..combi[0]-1]+facs[combi[1]..-1])
    words[1] = Word.new(facs[combi[0]..combi[1]-1])
    
    {divs: combi, linking: lk(myw, combi), words: words}
  end
  #-----
  
  if verbose
    result = monomials
  else
    monomials.delete_if{|mono| mono[:linking] == 0}
#    p monomials.map{|mono| mono[:words].map(&:show)}
    monomials.each{|mono| mono[:words] = mono[:words].map(&:cyclic_reduce)}
#    p monomials.map{|mono| mono[:words].map(&:show)}
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      pop[:coeff] = pop[:linking]
      #---
      monomials.each do |mono|
        if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
          pop[:coeff] += mono[:linking]
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
          pop[:coeff] += mono[:linking]*(-1)
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:linking].nil?}
    end 
  end
  return (result.empty?) ? [{divs: [], coeff: 0, words: []}] : result
end; nil

In [20]:
w = Word.new(a[1], a[2], a[3].inverse, b[1], b[1].inverse, a[3], a[1].inverse, a[2], a[2].inverse)

p w.show
#p w.contract_once.show
#p w.contract.show
p w.cyclic_reduce.show
nil

"acEbBeAcC"
"c"


# Execution

In [24]:
a1 = Word.new(a[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse)
  ]
  }

arr = samples #.slice(:s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each do |s|
    ws = s.show
    cob = cobracket(s, verbose: false).map do |h|
     coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
#       coeff = (h[:linking]==1) ? '' : "(#{h[:linking].to_s})"
      coeff + "#{h[:words].join(" \u{2227} ")}"
    end
    #---
    puts "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
  end
end;nil

--- s0 ---
 a  |--δ-->  (0)
a.d  |--δ-->  (-1)a ∧ d
a.D  |--δ-->  (0)
a.abA.aBA  |--δ-->  aB ∧ b + B ∧ ab + 1 ∧ a
--- s1 ---
 a.a.a.a  |--δ-->  aaa ∧ a + aa ∧ aa + a ∧ aaa
abA  |--δ-->  1 ∧ b
abAB  |--δ-->  (0)
b.a.b  |--δ-->  (0)
--- s2 ---
 a.dcDC  |--δ-->  (-1)a ∧ dcDC
a.aBA.dcDC.abA  |--δ-->  (-1)aB ∧ AdcDCab + 1 ∧ aBAdcDCab
a.dcDC.cdCDaBAdcDCabA  |--δ-->  (-1)aB ∧ AdcDCab + dcdCDaBAdcDCab ∧ D + (2)1 ∧ aBAdcDCab
aBAdcDCab  |--δ-->  (0)
--- s3 ---
 a.cdCD  |--δ-->  (0)
a.cdCD.cdCD  |--δ-->  acdCD ∧ cdCD
a.cdCD.abA.cdCD.aBA  |--δ-->  cdCDaBA ∧ cdCDab
--- s4 ---
 a.c.c.c  |--δ-->  acc ∧ c + ac ∧ cc
a.cD.cD.cD  |--δ-->  acDcD ∧ cD + acD ∧ cDcD
--- s5 ---
 a.ceCE  |--δ-->  (-1)aceC ∧ E + ace ∧ CE + (-1)ac ∧ C
a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  (-1)acdCDefEFceCfeFEdcDC ∧ E + acdCDefEFcefeFEdcDC ∧ CE + (-1)acdCDefEFcfeFEdcDC ∧ C
--- s6 ---
 a.bcBC  |--δ-->  (-1)abcB ∧ C + abc ∧ BC + (-1)abC ∧ cB
a.A.b.a  |--δ-->  (0)
abA.A.a  |--δ-->  (-1)ab ∧ A + a ∧ bA + 1 ∧ b
abA.aBA  |--δ-->  B ∧ b


## Experiments

In [27]:
mw = a1*c*(c.inverse) #Word.new(b[1], a[1], a[1].inverse, b[1].inverse, a[1]) #Word.new(a[1], b[1], b[1].inverse, a[2]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
printf mw.show + "\n-------\n"

vbs = true #false #
cobracket(mw, verbose: vbs).each do |h|
  h.each do |k,v| 
    v = v.map(&:cyclic_reduce).join(" \u{2227} ") if k == :words
    p "#{k}: #{v}"
  end
  printf "---\n"
end; nil

a.abA.aBA
-------
"divs: [1, 2]"
"linking: 1"
"words: 1 ∧ a"
---
"divs: [1, 3]"
"linking: 1"
"words: B ∧ ab"
---
"divs: [1, 4]"
"linking: 0"
"words: aB ∧ b"
---
"divs: [1, 5]"
"linking: 0"
"words: B ∧ ab"
---
"divs: [1, 6]"
"linking: 0"
"words: 1 ∧ a"
---
"divs: [1, 7]"
"linking: 0"
"words: a ∧ 1"
---
"divs: [2, 3]"
"linking: 1"
"words: aB ∧ b"
---
"divs: [2, 4]"
"linking: 0"
"words: aaB ∧ bA"
---
"divs: [2, 5]"
"linking: 0"
"words: aB ∧ b"
---
"divs: [2, 6]"
"linking: 0"
"words: a ∧ 1"
---
"divs: [2, 7]"
"linking: 0"
"words: aa ∧ A"
---
"divs: [3, 4]"
"linking: 0"
"words: abaB ∧ A"
---
"divs: [3, 5]"
"linking: 0"
"words: a ∧ 1"
---
"divs: [3, 6]"
"linking: 0"
"words: ab ∧ B"
---
"divs: [3, 7]"
"linking: 0"
"words: aab ∧ BA"
---
"divs: [4, 5]"
"linking: 0"
"words: abAB ∧ a"
---
"divs: [4, 6]"
"linking: 0"
"words: b ∧ aB"
---
"divs: [4, 7]"
"linking: 0"
"words: ab ∧ B"
---
"divs: [5, 6]"
"linking: 0"
"words: ab ∧ B"
---
"divs: [5, 7]"
"linking: 0"
"words: aab ∧ BA"
---
"divs: [6, 7]"
"l

In [31]:
mw = Word.new(b[2], b[1], a[1], a[1].inverse, b[1].inverse, a[1]) #Word.new(a[1], b[1], b[1].inverse, a[2]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
printf mw.show + "\n-------\n"

vbs = false #true #
cobracket(mw, verbose: vbs).each do |h|
  h.each do |k,v| 
    v = v.map(&:cyclic_reduce).join(" \u{2227} ") if k == :words
    p "#{k}: #{v}"
  end
  printf "---\n"
end; nil

dbaABa
-------
"divs: [[5, 6]]"
"linking: 1"
"words: d ∧ a"
"coeff: 1"
---


In [11]:
myw = samples[:s6][-1]
p myw.show

vb = false #true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

"abAaBA"
=====
[["aAaBA", "b"]]
[["B", "b"]]
"linking: 1"
"B ∧ b"
----------


### TODO

* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。
* [x] a.abA.aBA の cobraket にバグあり。
* [x] div 同士の linking を div に対応する2文字ワード同士の単純な計算で求められないか?  <-- **DONE!**
* [ ] Wcode#devide して得られる2つの Wcode instance の show(2) (および show(3)) がバグってる。なぜか each_slice(4) が2つずつくくる。
* [x] 係数がプラマイ逆かな。
* [x] simple subword たちによる計算を実装する。<-- ダメだった。各 simple subword が内部では確実に交差しない保証がないとダメだから。